In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import t
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error
from itertools import combinations
from math import comb

# Data Pre-Processing

In [ ]:
df1 = pd.read_csv('data_2017_2021.csv')
df2 = pd.read_csv('data_2022_2023.csv')

In [ ]:
#Fix names
df2.loc[(df2["FirstName"]== "Kate") & (df2["Competition"]=='2022 51st FIG Artistic Gymnastics World Championships'),["LastName"]] = "MCDONALD"
df2.loc[(df2["FirstName"]== "Kate") & (df2["Competition"]=='BIRMINGHAM 2022 Commonwealth Games'),["LastName"]] = "MCDONALD"
df2.loc[(df2["FirstName"]== "Daria Angelina") & (df2["Competition"]=='2023 Varna World Challenge Cup Results'),["LastName"]] = "HARTMANN"
df2.loc[(df2["LastName"]== "ABIYURAFI") & (df2["Competition"]=='FIG Apparatus World Cup 2023'),["FirstName"]] = "ABIYURAFI"
df2.loc[(df2["LastName"]== "CALVO MORENO JO") & (df2["Competition"]=='2022 51st FIG Artistic Gymnastics World Championships'),["FirstName"]] = "Jossimar Orlando"
df2.loc[(df2["FirstName"]== "Jossimar Orlando") & (df2["Competition"]=='2022 51st FIG Artistic Gymnastics World Championships'),["LastName"]] = "CALVO MORENO"
df2.loc[(df2["LastName"]== "ELPITIYA BADALG D") & (df2["Competition"]=='BIRMINGHAM 2022 Commonwealth Games'),["FirstName"]] = "G.D."
df2.loc[(df2["FirstName"]== "G.D.") & (df2["Competition"]=='BIRMINGHAM 2022 Commonwealth Games'),["LastName"]] = "ELPITIYA BADALG"
df2.loc[(df2["LastName"]== "KHOSRONEZHAD") & (df2["Competition"]=='FIG Apparatus World Cup 2023'),["FirstName"]] = "Mohammadreza"
df2.loc[(df2["LastName"]== "KUMARASINGHEGE HG") & (df2["Competition"]=='BIRMINGHAM 2022 Commonwealth Games'),["FirstName"]] = "HG"
df2.loc[(df2["FirstName"]== "HG") & (df2["Competition"]=='BIRMINGHAM 2022 Commonwealth Games'),["LastName"]] = "KUMARASINGHEGE"
df2.loc[(df2["LastName"]== "NOURISHOURAKCHALI") & (df2["Competition"]=='FIG Apparatus World Cup 2023'),["FirstName"]] = "Hossein"
df2.loc[(df2["LastName"]== "YONEKURA") & (df2["Competition"]=='FIG Apparatus World Cup 2023'),["FirstName"]] = "Hidenobu"

In [ ]:
df1.loc[(df1["LastName"]== "JESUS SANT") & (df1["Competition"]=='Olympic Games'),["FirstName"]] = "Melanie"
df1.loc[(df1["FirstName"]== "Melanie") & (df1["Competition"]=='Olympic Games'),["LastName"]] = "JESUS DOS SANTOS"

In [ ]:
df1

LastName  FirstName Gender Country             Date    Competition  \
0         ABDUL       HADI      w     MAS  SUN 25 JUL 2021  Olympic Games   
1         ABDUL       HADI      w     MAS  SUN 25 JUL 2021  Olympic Games   
2         ABDUL       HADI      w     MAS  SUN 25 JUL 2021  Olympic Games   
3         ABDUL       HADI      w     MAS  SUN 25 JUL 2021  Olympic Games   
4    ABDUL HADI  Farah Ann      w     MAS     25 July 2021  Olympic Games   
..          ...        ...    ...     ...              ...            ...   
760          de      JESUS      w     FRA  SUN 25 JUL 2021  Olympic Games   
761         van        POL      w     NED  SUN 25 JUL 2021  Olympic Games   
762         van        POL      w     NED  SUN 25 JUL 2021  Olympic Games   
763         van        POL      w     NED  SUN 25 JUL 2021  Olympic Games   
764         van        POL      w     NED  SUN 25 JUL 2021  Olympic Games   

    Round      Location Apparatus  Rank  D_Score  E_Score  Penalty   Score  
0    qual  Tokyo, Japan        BB  76.0      4.8    6.766      NaN  11.566  
1    qual  Tokyo, Japan        FX  64.0      4.6    7.633      NaN  12.233  
2    qual  Tokyo, Japan        UE  74.0      5.1    6.500      NaN  11.600  
3    qual  Tokyo, Japan        VT  72.0      4.6    8.566      NaN  13.166  
4    qual  Tokyo, Japan        BB  76.0      4.8    6.766      NaN  11.566  
..    ...           ...       ...   ...      ...      ...      ...     ...  
760  qual  Tokyo, Japan        VT  19.0      5.4    9.166      0.1  14.466  
761  qual  Tokyo, Japan        BB  75.0      4.7    6.900      NaN  11.600  
762  qual  Tokyo, Japan        FX  33.0      4.9    8.000      NaN  12.900  
763  qual  Tokyo, Japan        UE  41.0      5.3    7.833      NaN  13.133  
764  qual  Tokyo, Japan        VT  38.0      5.0    9.100      NaN  14.100  

[765 rows x 14 columns]

In [ ]:
df2

LastName FirstName Gender Country            Date  \
0          AAS   Fredrik      m     NOR  24-27 Feb 2022   
1          AAS   Fredrik      m     NOR  24-27 Feb 2022   
2          AAS   Fredrik      m     NOR  24-27 Feb 2022   
3          AAS   Fredrik      m     NOR  23-26 Feb 2023   
4          AAS   Fredrik      m     NOR  23-26 Feb 2023   
...        ...       ...    ...     ...             ...   
24429    ÖNDER     Ahmet      m     TUR    1-4 Jun 2023   
24430    ÖNDER     Ahmet      m     TUR    1-4 Jun 2023   
24431    ÖNDER     Ahmet      m     TUR    1-4 Jun 2023   
24432    ÖNDER     Ahmet      m     TUR    1-4 Jun 2023   
24433    ÖNDER     Ahmet      m     TUR    1-4 Jun 2023   

                             Competition  Round          Location Apparatus  \
0                 2022 Cottbus World Cup   qual  Cottbus, Germany        HB   
1                 2022 Cottbus World Cup   qual  Cottbus, Germany        PB   
2                 2022 Cottbus World Cup   qual  Cottbus, Germany        PH   
3                 2023 Cottbus World Cup   qual  Cottbus, Germany        HB   
4                 2023 Cottbus World Cup   qual  Cottbus, Germany        PH   
...                                  ...    ...               ...       ...   
24429  2023 Tel Aviv World Challenge Cup  final  Tel Aviv, Israel        FX   
24430  2023 Tel Aviv World Challenge Cup   qual  Tel Aviv, Israel        FX   
24431  2023 Tel Aviv World Challenge Cup   qual  Tel Aviv, Israel        HB   
24432  2023 Tel Aviv World Challenge Cup  final  Tel Aviv, Israel        PB   
24433  2023 Tel Aviv World Challenge Cup   qual  Tel Aviv, Israel        PB   

       Rank  D_Score  E_Score  Penalty   Score  
0      18.0      3.9    8.266      NaN  12.166  
1      23.0      3.9    6.900      NaN  10.800  
2      33.0      4.2    6.666      NaN  10.866  
3      39.0      4.6    6.700      NaN  11.300  
4      44.0      4.4    7.800      NaN  12.200  
...     ...      ...      ...      ...     ...  
24429   8.0      4.8    7.050      NaN  11.850  
24430   3.0      5.8    7.950      0.1  13.650  
24431  10.0      5.0    6.250      NaN  11.250  
24432   1.0      6.3    8.050      NaN  14.350  
24433   8.0      6.0    7.400      NaN  13.400  

[24434 rows x 14 columns]

In [ ]:
# concat data
df = pd.concat([df1, df2], ignore_index=True)
df

LastName  FirstName Gender Country             Date  \
0           ABDUL       HADI      w     MAS  SUN 25 JUL 2021   
1           ABDUL       HADI      w     MAS  SUN 25 JUL 2021   
2           ABDUL       HADI      w     MAS  SUN 25 JUL 2021   
3           ABDUL       HADI      w     MAS  SUN 25 JUL 2021   
4      ABDUL HADI  Farah Ann      w     MAS     25 July 2021   
...           ...        ...    ...     ...              ...   
25194       ÖNDER      Ahmet      m     TUR     1-4 Jun 2023   
25195       ÖNDER      Ahmet      m     TUR     1-4 Jun 2023   
25196       ÖNDER      Ahmet      m     TUR     1-4 Jun 2023   
25197       ÖNDER      Ahmet      m     TUR     1-4 Jun 2023   
25198       ÖNDER      Ahmet      m     TUR     1-4 Jun 2023   

                             Competition  Round          Location Apparatus  \
0                          Olympic Games   qual      Tokyo, Japan        BB   
1                          Olympic Games   qual      Tokyo, Japan        FX   
2                          Olympic Games   qual      Tokyo, Japan        UE   
3                          Olympic Games   qual      Tokyo, Japan        VT   
4                          Olympic Games   qual      Tokyo, Japan        BB   
...                                  ...    ...               ...       ...   
25194  2023 Tel Aviv World Challenge Cup  final  Tel Aviv, Israel        FX   
25195  2023 Tel Aviv World Challenge Cup   qual  Tel Aviv, Israel        FX   
25196  2023 Tel Aviv World Challenge Cup   qual  Tel Aviv, Israel        HB   
25197  2023 Tel Aviv World Challenge Cup  final  Tel Aviv, Israel        PB   
25198  2023 Tel Aviv World Challenge Cup   qual  Tel Aviv, Israel        PB   

       Rank  D_Score  E_Score  Penalty   Score  
0      76.0      4.8    6.766      NaN  11.566  
1      64.0      4.6    7.633      NaN  12.233  
2      74.0      5.1    6.500      NaN  11.600  
3      72.0      4.6    8.566      NaN  13.166  
4      76.0      4.8    6.766      NaN  11.566  
...     ...      ...      ...      ...     ...  
25194   8.0      4.8    7.050      NaN  11.850  
25195   3.0      5.8    7.950      0.1  13.650  
25196  10.0      5.0    6.250      NaN  11.250  
25197   1.0      6.3    8.050      NaN  14.350  
25198   8.0      6.0    7.400      NaN  13.400  

[25199 rows x 14 columns]

In [ ]:
def eda(df):
    # delete rows whose penaly is greater than 1
    df = df[~(df['Penalty']>1)]

    # delete rows whose both D_Score and E_Score are zero
    df = df[~((df['D_Score']==0) & (df['E_Score']==0))]

    # round name
    df['Round'] = df['Round'].replace(['AAqual', 'TeamQual'], 'qual')
    df['Round'] = df['Round'].replace(['AAfinal', 'TeamFinal'], 'final')

    # VT, HB apparatus name
    df['Apparatus'] = df['Apparatus'].replace(['VT1', 'VT2'], 'VT')
    df['Apparatus'] = df['Apparatus'].replace('hb', 'HB')

    # make Full name column
    df['Name'] = df['FirstName'] + ' ' + df['LastName']

    # split by round
    df_final = df[df['Round']=='final'].reset_index()
    df_qual = df[df['Round']=='qual'].reset_index()

    # labelling of qualification
    df_same = df_qual[df_qual.duplicated(subset=['Competition', 'Name', 'Apparatus'], keep=False)]
    different_round = df_same[df_same.duplicated(subset=['Competition', 'Name', 'Apparatus', 'Round'], keep=False)].reset_index()
    different_round['Label'] = 1
    qual_label = pd.merge(df_qual, different_round, how='left')
    qual_label['Label'] = qual_label['Label'].fillna(0)

    # split by gender
    male_qual = qual_label[qual_label['Gender']=='m']
    female_qual = qual_label[qual_label['Gender']=='w']
    male_final = df_final[df_final['Gender']=='m']
    female_final = df_final[df_final['Gender']=='w']

    return male_qual, male_final, female_qual, female_final

In [ ]:
male_qual, male_final, female_qual, female_final = eda(df)

In [ ]:
# split by apparatus

def male(df):
    m_HB = df[df['Apparatus']=='HB'].reset_index(drop=True)
    m_PB = df[df['Apparatus']=='PB'].reset_index(drop=True)
    m_PH = df[df['Apparatus']=='PH'].reset_index(drop=True)
    m_FX = df[df['Apparatus']=='FX'].reset_index(drop=True)
    m_SR = df[df['Apparatus']=='SR'].reset_index(drop=True)
    m_VT = df[df['Apparatus']=='VT'].reset_index(drop=True)

    return m_HB, m_PB, m_PH, m_FX, m_SR, m_VT

def female(df):
    w_UB = df[df['Apparatus']=='UB'].reset_index(drop=True)
    w_BB = df[df['Apparatus']=='BB'].reset_index(drop=True)
    w_VT = df[df['Apparatus']=='VT'].reset_index(drop=True)
    w_FX = df[df['Apparatus']=='FX'].reset_index(drop=True)

    return w_UB, w_BB, w_VT, w_FX

In [ ]:
m_HB_q, m_PB_q, m_PH_q, m_FX_q, m_SR_q, m_VT_q = male(male_qual)
m_HB_f, m_PB_f, m_PH_f, m_FX_f, m_SR_f, m_VT_f = male(male_final)

w_UB_q, w_BB_q, w_VT_q, w_FX_q = female(female_qual)
w_UB_f, w_BB_f, w_VT_f, w_FX_f = female(female_final)

# Test of Host Country Advantage

In [ ]:
#whole : concated data
whole = pd.concat([male_qual, male_final, female_qual, female_final], ignore_index=True)
whole = whole.drop(0)
whole = whole.reset_index(drop=True)
whole

index LastName  FirstName Gender Country            Date  \
0        766      AAS    Fredrik      m     NOR  24-27 Feb 2022   
1        767      AAS    Fredrik      m     NOR  24-27 Feb 2022   
2        768      AAS    Fredrik      m     NOR  23-26 Feb 2023   
3        769      AAS    Fredrik      m     NOR  23-26 Feb 2023   
4        770      AAS    Fredrik      m     NOR    1-5 Aug 2023   
...      ...      ...        ...    ...     ...             ...   
25074  25162      ZUO       Tong      w     CHN  24-29 Sep 2023   
25075  25163      ZUO       Tong      w     CHN  24-29 Sep 2023   
25076  25165      ZUO       Tong      w     CHN  24-29 Sep 2023   
25077  25166      ZUO       Tong      w     CHN  24-29 Sep 2023   
25078  25171  ZWICKER  Christina      w     CRO   8-11 Jun 2023   

                            Competition  Round          Location Apparatus  \
0                2022 Cottbus World Cup   qual  Cottbus, Germany        PB   
1                2022 Cottbus World Cup   qual  Cottbus, Germany        PH   
2                2023 Cottbus World Cup   qual  Cottbus, Germany        HB   
3                2023 Cottbus World Cup   qual  Cottbus, Germany        PH   
4      2023 FISU World University Games   qual    Chengdu, China        FX   
...                                 ...    ...               ...       ...   
25074    HANGZHOU 2022 19th Asian Games  final   Hangzhou, China        FX   
25075    HANGZHOU 2022 19th Asian Games  final   Hangzhou, China        UB   
25076    HANGZHOU 2022 19th Asian Games  final   Hangzhou, China        UB   
25077    HANGZHOU 2022 19th Asian Games  final   Hangzhou, China        VT   
25078   2023 Osijek World Challenge Cup  final   Osijek, Croatia        BB   

       Rank  D_Score  E_Score  Penalty   Score               Name  level_0  \
0      23.0      3.9    6.900      NaN  10.800        Fredrik AAS      NaN   
1      33.0      4.2    6.666      NaN  10.866        Fredrik AAS      NaN   
2      39.0      4.6    6.700      NaN  11.300        Fredrik AAS      NaN   
3      44.0      4.4    7.800      NaN  12.200        Fredrik AAS      NaN   
4      54.0      4.0    8.566      NaN  12.566        Fredrik AAS      NaN   
...     ...      ...      ...      ...     ...                ...      ...   
25074   2.0      5.1    8.166      0.1  13.166           Tong ZUO      NaN   
25075   3.0      5.8    8.066      NaN  13.866           Tong ZUO      NaN   
25076   1.0      6.0    7.733      NaN  13.733           Tong ZUO      NaN   
25077   5.0      4.2    8.633      NaN  12.833           Tong ZUO      NaN   
25078   5.0      4.7    7.966      NaN  12.666  Christina ZWICKER      NaN   

       Label  
0        0.0  
1        0.0  
2        0.0  
3        0.0  
4        0.0  
...      ...  
25074    NaN  
25075    NaN  
25076    NaN  
25077    NaN  
25078    NaN  

[25079 rows x 18 columns]

In [ ]:
# read excel file that contains country name and abb (https://www.olympiandatabase.com/index.php?id=1670&L=1#google_vignette)
country_df = pd.read_excel("country_abb.xls")
#country_df

In [ ]:
# convert it to dictionary
country_abb = dict(zip(country_df['ABB'], country_df['Country']))
country_abb["nan"] = ""
#country_abb

In [ ]:
whole["Location_Country"] = ""
whole["Country_Country"] = ""

In [ ]:
for i in range(0, whole.shape[0]):
    if len(whole.at[i, "Location"].split(",")) > 1:
        whole.at[i, "Location_Country"] = whole.at[i, "Location"].split(",")[1].lstrip()
    else:
        whole.at[i, "Location_Country"] = whole.at[i, "Location"].strip()
    whole.at[i, "Country_Country"] = country_abb[str(whole.at[i, "Country"])]

In [ ]:
whole

index LastName  FirstName Gender Country            Date  \
0        766      AAS    Fredrik      m     NOR  24-27 Feb 2022   
1        767      AAS    Fredrik      m     NOR  24-27 Feb 2022   
2        768      AAS    Fredrik      m     NOR  23-26 Feb 2023   
3        769      AAS    Fredrik      m     NOR  23-26 Feb 2023   
4        770      AAS    Fredrik      m     NOR    1-5 Aug 2023   
...      ...      ...        ...    ...     ...             ...   
25074  25162      ZUO       Tong      w     CHN  24-29 Sep 2023   
25075  25163      ZUO       Tong      w     CHN  24-29 Sep 2023   
25076  25165      ZUO       Tong      w     CHN  24-29 Sep 2023   
25077  25166      ZUO       Tong      w     CHN  24-29 Sep 2023   
25078  25171  ZWICKER  Christina      w     CRO   8-11 Jun 2023   

                            Competition  Round          Location Apparatus  \
0                2022 Cottbus World Cup   qual  Cottbus, Germany        PB   
1                2022 Cottbus World Cup   qual  Cottbus, Germany        PH   
2                2023 Cottbus World Cup   qual  Cottbus, Germany        HB   
3                2023 Cottbus World Cup   qual  Cottbus, Germany        PH   
4      2023 FISU World University Games   qual    Chengdu, China        FX   
...                                 ...    ...               ...       ...   
25074    HANGZHOU 2022 19th Asian Games  final   Hangzhou, China        FX   
25075    HANGZHOU 2022 19th Asian Games  final   Hangzhou, China        UB   
25076    HANGZHOU 2022 19th Asian Games  final   Hangzhou, China        UB   
25077    HANGZHOU 2022 19th Asian Games  final   Hangzhou, China        VT   
25078   2023 Osijek World Challenge Cup  final   Osijek, Croatia        BB   

       Rank  D_Score  E_Score  Penalty   Score               Name  level_0  \
0      23.0      3.9    6.900      NaN  10.800        Fredrik AAS      NaN   
1      33.0      4.2    6.666      NaN  10.866        Fredrik AAS      NaN   
2      39.0      4.6    6.700      NaN  11.300        Fredrik AAS      NaN   
3      44.0      4.4    7.800      NaN  12.200        Fredrik AAS      NaN   
4      54.0      4.0    8.566      NaN  12.566        Fredrik AAS      NaN   
...     ...      ...      ...      ...     ...                ...      ...   
25074   2.0      5.1    8.166      0.1  13.166           Tong ZUO      NaN   
25075   3.0      5.8    8.066      NaN  13.866           Tong ZUO      NaN   
25076   1.0      6.0    7.733      NaN  13.733           Tong ZUO      NaN   
25077   5.0      4.2    8.633      NaN  12.833           Tong ZUO      NaN   
25078   5.0      4.7    7.966      NaN  12.666  Christina ZWICKER      NaN   

       Label Location_Country Country_Country  
0        0.0          Germany          Norway  
1        0.0          Germany          Norway  
2        0.0          Germany          Norway  
3        0.0          Germany          Norway  
4        0.0            China          Norway  
...      ...              ...             ...  
25074    NaN            China           China  
25075    NaN            China           China  
25076    NaN            China           China  
25077    NaN            China           China  
25078    NaN          Croatia         Croatia  

[25079 rows x 20 columns]

In [ ]:
host = whole[whole["Location_Country"] == whole["Country_Country"]]
guest = whole[whole["Location_Country"] != whole["Country_Country"]]
#host

In [ ]:
host_score = np.array(host["Score"].dropna())
guest_score = np.array(guest["Score"].dropna())
t_stat, p_value = stats.ttest_ind(host_score, guest_score, equal_var=False) #two-sample t-test
print("host sample size, mean, var:", len(host_score), host_score.mean(), host_score.var())
print("guest sample size, mean, var:", len(guest_score), guest_score.mean(), guest_score.var())
print("difference:", host_score.mean() - guest_score.mean())
print("t-statistic:", t_stat)
print("p-value:", p_value)
# as the p-value is very small, we can conclude that there is a significant difference between the two groups

host sample size, mean, var: 1178 12.965927843803055 1.4489886900396631
guest sample size, mean, var: 23895 12.762781042059007 1.6435976942629025
difference: 0.20314680174404742
t-statistic: 5.634552809391613
p-value: 2.1454869999272184e-08


# Ridge Regression

In [ ]:
whole["full_name"] = ""
whole["id"] = ""

for i in range(whole.shape[0]):
    if pd.notna(whole.at[i, "LastName"]):  # "LastName" is not NaN
        whole.at[i, "full_name"] = str(whole.at[i, "FirstName"]).lower() + "_" + str(whole.at[i, "LastName"]).lower()
    else: # "LastName" is NaN
        whole.at[i, "full_name"] = str(whole.at[i, "FirstName"]).lower()

for i in range(0, whole.shape[0]):
    whole.at[i, "id"] = whole.at[i, "Country_Country"] + "_" + whole.at[i, "full_name"] #+ "_" + whole.at[i, "Apparatus"]

In [ ]:
whole

index LastName  FirstName Gender Country            Date  \
0        766      AAS    Fredrik      m     NOR  24-27 Feb 2022   
1        767      AAS    Fredrik      m     NOR  24-27 Feb 2022   
2        768      AAS    Fredrik      m     NOR  23-26 Feb 2023   
3        769      AAS    Fredrik      m     NOR  23-26 Feb 2023   
4        770      AAS    Fredrik      m     NOR    1-5 Aug 2023   
...      ...      ...        ...    ...     ...             ...   
25074  25162      ZUO       Tong      w     CHN  24-29 Sep 2023   
25075  25163      ZUO       Tong      w     CHN  24-29 Sep 2023   
25076  25165      ZUO       Tong      w     CHN  24-29 Sep 2023   
25077  25166      ZUO       Tong      w     CHN  24-29 Sep 2023   
25078  25171  ZWICKER  Christina      w     CRO   8-11 Jun 2023   

                            Competition  Round          Location Apparatus  \
0                2022 Cottbus World Cup   qual  Cottbus, Germany        PB   
1                2022 Cottbus World Cup   qual  Cottbus, Germany        PH   
2                2023 Cottbus World Cup   qual  Cottbus, Germany        HB   
3                2023 Cottbus World Cup   qual  Cottbus, Germany        PH   
4      2023 FISU World University Games   qual    Chengdu, China        FX   
...                                 ...    ...               ...       ...   
25074    HANGZHOU 2022 19th Asian Games  final   Hangzhou, China        FX   
25075    HANGZHOU 2022 19th Asian Games  final   Hangzhou, China        UB   
25076    HANGZHOU 2022 19th Asian Games  final   Hangzhou, China        UB   
25077    HANGZHOU 2022 19th Asian Games  final   Hangzhou, China        VT   
25078   2023 Osijek World Challenge Cup  final   Osijek, Croatia        BB   

       ...  E_Score  Penalty   Score               Name  level_0 Label  \
0      ...    6.900      NaN  10.800        Fredrik AAS      NaN   0.0   
1      ...    6.666      NaN  10.866        Fredrik AAS      NaN   0.0   
2      ...    6.700      NaN  11.300        Fredrik AAS      NaN   0.0   
3      ...    7.800      NaN  12.200        Fredrik AAS      NaN   0.0   
4      ...    8.566      NaN  12.566        Fredrik AAS      NaN   0.0   
...    ...      ...      ...     ...                ...      ...   ...   
25074  ...    8.166      0.1  13.166           Tong ZUO      NaN   NaN   
25075  ...    8.066      NaN  13.866           Tong ZUO      NaN   NaN   
25076  ...    7.733      NaN  13.733           Tong ZUO      NaN   NaN   
25077  ...    8.633      NaN  12.833           Tong ZUO      NaN   NaN   
25078  ...    7.966      NaN  12.666  Christina ZWICKER      NaN   NaN   

       Location_Country  Country_Country          full_name  \
0               Germany           Norway        fredrik_aas   
1               Germany           Norway        fredrik_aas   
2               Germany           Norway        fredrik_aas   
3               Germany           Norway        fredrik_aas   
4                 China           Norway        fredrik_aas   
...                 ...              ...                ...   
25074             China            China           tong_zuo   
25075             China            China           tong_zuo   
25076             China            China           tong_zuo   
25077             China            China           tong_zuo   
25078           Croatia          Croatia  christina_zwicker   

                              id  
0             Norway_fredrik_aas  
1             Norway_fredrik_aas  
2             Norway_fredrik_aas  
3             Norway_fredrik_aas  
4             Norway_fredrik_aas  
...                          ...  
25074             China_tong_zuo  
25075             China_tong_zuo  
25076             China_tong_zuo  
25077             China_tong_zuo  
25078  Croatia_christina_zwicker  

[25079 rows x 22 columns]

In [ ]:
# sort data by "Score" and "D_Score"
whole = whole.sort_values(by=['Score', 'D_Score'], ascending=[False, False])

# make "Adjusted Rank" column
whole['Adjusted Rank'] = whole.groupby(['Competition', 'Round', 'Apparatus']).cumcount() + 1

whole.head()

In [ ]:
whole["Home_advantage"] = (whole["Location_Country"] == whole["Country_Country"]).astype(int)

In [ ]:
whole.to_csv("whole.csv") # Pre-Processed data

In [ ]:
#whole_w : women qual and final
whole_w = whole[whole["Gender"] == "w"]

In [ ]:
american = list(set([value for value in whole_w["id"] if value.startswith("United States of America")]))
len(american)

62

In [ ]:
american.append("Medals")
design_matrix = pd.DataFrame(columns = american)
design_matrix

Empty DataFrame
Columns: [United States of America_annalisa_milton, United States of America_elle_mueller, United States of America_konnor_mcclain, United States of America_camryn_richardson, United States of America_olivia_greaves, United States of America_ella_murphy, United States of America_leigh_anne elliott, United States of America_mykayla_skinner, United States of America_charlotte_booth, United States of America_ciena_alipio, United States of America_madray_johnson, United States of America_kayla_dicello, United States of America_brooke_pierson, United States of America_alicia_zhou, United States of America_joscelyn_roberson, United States of America_nola_matthews, United States of America_lexi_zeiss, United States of America_eveylynn_lowe, United States of America_michelle_pineda, United States of America_chloe_cho, United States of America_gabriella_van frayen, United States of America_brynn_torry, United States of America_mc grace_callum, United States of America_gabby_disidore, United States of America_leanne_wong, United States of America_callum_mc, United States of America_kelise_woolford, United States of America_sydney_barros, United States of America_adriana_consoli, United States of America_sunisa_lee, United States of America_simone_biles, United States of America_karis_german, United States of America_amari_drayton, United States of America_marissa_neal, United States of America_norah_christian, United States of America_avery_king, United States of America_katelyn_rosen, United States of America_levi_jung-ruivivar, United States of America_lauren_little, United States of America_christiane_popovich, United States of America_myli_lew, United States of America_shilese_jones, United States of America_avery_moll, United States of America_jordan_chiles, United States of America_kaliya_lincoln, United States of America_dulcy_caylor, United States of America_addison_fatta, United States of America_jordis_eichman, United States of America_amelia_disidore, United States of America_katelyn_jong, United States of America_zoe_miller, United States of America_ashlee_sullivan, United States of America_skye_blakely, United States of America_nola rhianne_matthews, United States of America_tiana_sumanasekera, United States of America_jade_carey, United States of America_emjae_frazier, United States of America_joscelyn michelle_roberson, United States of America_skylar_draser, United States of America_paityn_walker, United States of America_malea_milton, United States of America_kaylen_morgan, Medals]
Index: []

[0 rows x 63 columns]

In [ ]:
competitions = whole_w["Competition"].unique()
competitions

array(['2023 U.S. Championships', '2023 Core Hydration Classic',
       'Olympic Games', 'SANTIAGO 2023 XIX Pan American Games',
       '2023 52nd FIG Artistic Gymnastics World Championships',
       '2023 10th Senior Artistic Gymnastics Asian Championships',
       '2022 51st FIG Artistic Gymnastics World Championships',
       '2022 9th Senior Artistic Gymnastics Asian Championships',
       'EnBW DTB Pokal Team Challenge 2023', '2022 U.S. Championships',
       '2023 Doha World Cup', '2023 Baku World Cup',
       '2022 Paris World Challenge Cup', '2022 Doha World Cup',
       '2022 Senior European Championships', '2023 Cairo World Cup',
       '2022 Winter Cup', '2022 British Gymnastics Championships',
       '2022 Baku World Cup', '2023 Osijek World Challenge Cup',
       '2022 U.S. Classic', '2023 Cottbus World Cup',
       '2023 Senior European Championships',
       'HANGZHOU 2022 19th Asian Games',
       '2023 FISU World University Games',
       '2022 Koper World Challenge Cu

In [ ]:
competition_dict = dict()
for compet in competitions:
    data = whole_w[whole_w["Competition"] == compet]
    lst = data["id"].unique()
    lst = list(set([value for value in lst if value.startswith("United States of America")]))
    data = data[data["Round"] != "qual"]
    data['Medal'] = np.where(data['Adjusted Rank'].isin([1.0,2.0,3.0]), 1, 0)
    medals = sum(data[data["id"].str.startswith("United States of America")]["Medal"])
    if compet in ["", ""]: #team medals 추가 올림픽,
        medals += dic[compet]
    #if tema_medal:
    #    medals += 1
    lst.append(medals)
    competition_dict[compet] = lst

In [ ]:
competition_dict

{'2023 U.S. Championships': ['United States of America_charlotte_booth',
  'United States of America_madray_johnson',
  'United States of America_kayla_dicello',
  'United States of America_alicia_zhou',
  'United States of America_joscelyn_roberson',
  'United States of America_nola_matthews',
  'United States of America_eveylynn_lowe',
  'United States of America_lexi_zeiss',
  'United States of America_michelle_pineda',
  'United States of America_leanne_wong',
  'United States of America_kelise_woolford',
  'United States of America_sunisa_lee',
  'United States of America_simone_biles',
  'United States of America_marissa_neal',
  'United States of America_levi_jung-ruivivar',
  'United States of America_myli_lew',
  'United States of America_shilese_jones',
  'United States of America_jordan_chiles',
  'United States of America_kaliya_lincoln',
  'United States of America_dulcy_caylor',
  'United States of America_addison_fatta',
  'United States of America_amelia_disidore',
  'U

In [ ]:
team_medal = pd.read_csv('team_medal.csv')
team_medal

Unnamed: 0                      Competition  Women  Men
0           0               2020-olympic-games      1    0
1           1         2023-world-championships      1    1
2           2         2022-world-championships      1    0
3           3  2023-pan-american-championships      1    1
4           4         2023-dtb-pokal-mixed-cup      0    0

In [ ]:
team_compet = {'Olympic Games' : "2020-olympic-games", '2023 52nd FIG Artistic Gymnastics World Championships' : "2023-world-championships",
               '2022 51st FIG Artistic Gymnastics World Championships' : "2022-world-championships",
               '2023 Artistic Gymnastics Senior Pan American Championships' : "2023-pan-american-championships"}

In [ ]:
i = 0
for compet in competitions:
    lst = competition_dict[compet]
    if len(lst) < 2:
        continue
    players = lst[0:-1]
    medals = lst[-1]
    for player in players:
        design_matrix.at[i, player] = 1
    if compet in team_compet.keys():
        medals += int(team_medal[team_medal["Competition"] == team_compet[compet]]["Women"])
    design_matrix.at[i, "Medals"] = medals
    i += 1
design_matrix = design_matrix.fillna(0)
design_matrix

United States of America_annalisa_milton  \
0                                          0   
1                                          1   
2                                          0   
3                                          0   
4                                          0   
5                                          0   
6                                          0   
7                                          0   
8                                          0   
9                                          0   
10                                         0   
11                                         0   
12                                         0   
13                                         1   
14                                         0   

    United States of America_elle_mueller  \
0                                       0   
1                                       1   
2                                       0   
3                                       0   
4                                       0   
5                                       0   
6                                       0   
7                                       1   
8                                       0   
9                                       0   
10                                      1   
11                                      0   
12                                      0   
13                                      1   
14                                      0   

    United States of America_konnor_mcclain  \
0                                         0   
1                                         0   
2                                         0   
3                                         0   
4                                         0   
5                                         0   
6                                         0   
7                                         1   
8                                         0   
9                                         0   
10                                        1   
11                                        0   
12                                        0   
13                                        0   
14                                        0   

    United States of America_camryn_richardson  \
0                                            0   
1                                            0   
2                                            0   
3                                            0   
4                                            0   
5                                            0   
6                                            0   
7                                            0   
8                                            0   
9                                            0   
10                                           1   
11                                           0   
12                                           0   
13                                           0   
14                                           0   

    United States of America_olivia_greaves  \
0                                         0   
1                                         0   
2                                         0   
3                                         0   
4                                         0   
5                                         0   
6                                         0   
7                                         1   
8                                         0   
9                                         0   
10                                        0   
11                                        0   
12                                        0   
13                                        0   
14                                        0   

    United States of America_ella_murphy  \
0                                      0   
1                                      1   
2                                      0   
3                              

In [ ]:
design_matrix.to_csv("design_matrix.csv")

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import RidgeCV

X = design_matrix.drop("Medals", axis=1).values
Y = design_matrix["Medals"].values

# train Ridge regression model
alphas = np.logspace(-6, 6, 10000)  # range of alpha
ridge_cv = RidgeCV(alphas=alphas, store_cv_values=True) #k-fold cross validation
ridge_cv.fit(X, Y)

# optimal alpha
best_alpha = ridge_cv.alpha_

# train Ridgre regression model with optimal alpha
ridge_model = RidgeCV(alphas=[best_alpha])
ridge_model.fit(X, Y)

RidgeCV(alphas=array([2.86976137]))

In [ ]:
# check results
print("Lambda", best_alpha)
coefficients_dic= {'Players': list(set([value for value in whole_w["id"] if value.startswith("United States of America")])),
                   'Beta': ridge_model.coef_}
coefficients = pd.DataFrame(coefficients_dic)
print("Coefficients:", coefficients)
print("Intercept:", ridge_model.intercept_)
y_pred = ridge_model.predict(X)
mse = mean_squared_error(Y, y_pred)
print(f"Mean Squared Error: {mse}")

Lambda 2.8697613656589205
Coefficients:                                               Players      Beta
0            United States of America_annalisa_milton  0.006075
1               United States of America_elle_mueller  0.194177
2             United States of America_konnor_mcclain  0.188102
3          United States of America_camryn_richardson  0.056845
4             United States of America_olivia_greaves  0.131257
..                                                ...       ...
57  United States of America_joscelyn michelle_rob...  0.627760
58             United States of America_skylar_draser  0.056845
59             United States of America_paityn_walker  0.056845
60              United States of America_malea_milton  0.006075
61             United States of America_kaylen_morgan  0.056845

[62 rows x 2 columns]
Intercept: 1.9290004388928752
Mean Squared Error: 0.5329705133444796


In [ ]:
coefficients
coefficients.sort_values(by = "Beta")
coefficients.sort_values(by = "Beta", ascending = False)

Players      Beta
10            United States of America_madray_johnson  0.762202
53     United States of America_nola rhianne_matthews  0.627760
57  United States of America_joscelyn michelle_rob...  0.627760
54        United States of America_tiana_sumanasekera  0.627065
46             United States of America_addison_fatta  0.624690
..                                                ...       ...
32             United States of America_amari_drayton -0.008578
14         United States of America_joscelyn_roberson -0.050967
51           United States of America_ashlee_sullivan -0.095720
50                United States of America_zoe_miller -0.207626
49              United States of America_katelyn_jong -0.261724

[62 rows x 2 columns]

In [ ]:
#reference : "https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-12-372" - Method - Significance testing in linear ridge regression
#t-test
#H0: beta = 0

X = design_matrix.drop("Medals", axis=1).values
Y = design_matrix["Medals"].values
n, p = X.shape
# beta_hat: (X'X + alpha*I)^(-1)X'Y
ridge_matrix = np.dot(X.T, X) + best_alpha * np.identity(p)
beta_hat = np.linalg.inv(ridge_matrix).dot(X.T).dot(Y)

inv_term = np.linalg.inv(ridge_matrix)
# H: H = X (X'X + lambda*I)^(-1)X'
H = np.dot(np.dot(X, inv_term), X.T)

v = n - np.trace(2 * H - np.dot(H, H.T)) #df

residuals = Y - np.dot(X, beta_hat)
sigma_hat_squared = np.dot(residuals.T, residuals) / v

# var_hat: var_hat = sigma_hat_squared * (X'X + lambda*I)^(-1) X'X (X'X + lambda*I)^(-1)
var_hat = sigma_hat_squared * np.dot(np.dot(inv_term, np.dot(X.T, X)), inv_term)

var_diagonal = np.diagonal(var_hat)
se_hat = np.sqrt(var_diagonal)

t_statistics = beta_hat / se_hat

from scipy.stats import t
# p-value:
#p_values = t.sf(np.abs(t_statistics), v) : P(T(v) > t_statistics)
p_values = t.sf(t_statistics, v)
#print(p_values)
print(p_values[p_values < 0.05]) #reject H0 #beta != 0
print(len(p_values[p_values < 0.05]), len(p_values))

[0.01365197 0.01365197 0.0166644  0.02145183 0.0189088 ]
5 62


In [ ]:
# get women players beta
whole_w = whole[whole["Gender"] == "w"]

american = list(set([value for value in whole_w["id"] if value.startswith("United States of America")]))

american.append("Medals")
design_matrix_w = pd.DataFrame(columns = american)

competitions = whole_w["Competition"].unique()

competition_dict = dict()
for compet in competitions:
    data = whole_w[whole_w["Competition"] == compet]
    lst = data["id"].unique()
    lst = list(set([value for value in lst if value.startswith("United States of America")]))
    data = data[data["Round"] != "qual"]
    data['Medal'] = np.where(data['Adjusted Rank'].isin([1.0,2.0,3.0]), 1, 0)
    medals = sum(data[data["id"].str.startswith("United States of America")]["Medal"])
    lst.append(medals)
    competition_dict[compet] = lst

i = 0
for compet in competitions:
    lst = competition_dict[compet]
    if len(lst) < 2:
        continue
    players = lst[0:-1]
    medals = lst[-1]
    for player in players:
        design_matrix_w.at[i, player] = 1
    if compet in team_compet.keys():
        medals += int(team_medal[team_medal["Competition"] == team_compet[compet]]["Women"])
    design_matrix_w.at[i, "Medals"] = medals
    i += 1
design_matrix_w = design_matrix_w.fillna(0)

X = design_matrix_w.drop("Medals", axis=1).values
Y = design_matrix_w["Medals"].values

alphas = np.logspace(-6, 6, 10000)
ridge_cv_w = RidgeCV(alphas=alphas, store_cv_values=True)
ridge_cv_w.fit(X, Y)

best_alpha = ridge_cv_w.alpha_

ridge_model_w = RidgeCV(alphas=[best_alpha])
ridge_model_w.fit(X, Y)

print("Lambda", best_alpha)
coefficients_dic_w = {'Players': list(set([value for value in whole_w["id"] if value.startswith("United States of America")])),
                   'Beta': ridge_model_w.coef_}
coefficients_w = pd.DataFrame(coefficients_dic_w)
print("Coefficients:", coefficients_w)
print("Intercept:", ridge_model_w.intercept_)
y_pred = ridge_model_w.predict(X)
mse = mean_squared_error(Y, y_pred)
print(f"Mean Squared Error: {mse}")

Lambda 2.8697613656589205
Coefficients:                                               Players      Beta
0            United States of America_annalisa_milton  0.006075
1               United States of America_elle_mueller  0.194177
2             United States of America_konnor_mcclain  0.188102
3          United States of America_camryn_richardson  0.056845
4             United States of America_olivia_greaves  0.131257
..                                                ...       ...
57  United States of America_joscelyn michelle_rob...  0.627760
58             United States of America_skylar_draser  0.056845
59             United States of America_paityn_walker  0.056845
60              United States of America_malea_milton  0.006075
61             United States of America_kaylen_morgan  0.056845

[62 rows x 2 columns]
Intercept: 1.9290004388928752
Mean Squared Error: 0.5329705133444796


In [ ]:
# get men players beta
whole_m = whole[whole["Gender"] == "m"]

american = list(set([value for value in whole_m["id"] if value.startswith("United States of America")]))

american.append("Medals")
design_matrix_m = pd.DataFrame(columns = american)

competitions = whole_m["Competition"].unique()

competition_dict = dict()
for compet in competitions:
    data = whole_m[whole_m["Competition"] == compet]
    lst = data["id"].unique()
    lst = list(set([value for value in lst if value.startswith("United States of America")]))
    data = data[data["Round"] != "qual"]
    data['Medal'] = np.where(data['Adjusted Rank'].isin([1.0,2.0,3.0]), 1, 0)
    medals = sum(data[data["id"].str.startswith("United States of America")]["Medal"])
    lst.append(medals)
    competition_dict[compet] = lst

i = 0
for compet in competitions:
    lst = competition_dict[compet]
    if len(lst) < 2:
        continue
    players = lst[0:-1]
    medals = lst[-1]
    for player in players:
        design_matrix_m.at[i, player] = 1
    if compet in team_compet.keys():
        medals += int(team_medal[team_medal["Competition"] == team_compet[compet]]["Men"])
    design_matrix_m.at[i, "Medals"] = medals
    i += 1
design_matrix_m = design_matrix_m.fillna(0)

X = design_matrix_m.drop("Medals", axis=1).values
Y = design_matrix_m["Medals"].values

alphas = np.logspace(-6, 6, 10000)
ridge_cv_m = RidgeCV(alphas=alphas, store_cv_values=True)
ridge_cv_m.fit(X, Y)

best_alpha = ridge_cv_m.alpha_

ridge_model_m = RidgeCV(alphas=[best_alpha])
ridge_model_m.fit(X, Y)

print("Lambda", best_alpha)
coefficients_dic_m= {'Players': list(set([value for value in whole_m["id"] if value.startswith("United States of America")])),
                   'Beta': ridge_model_m.coef_}
coefficients_m = pd.DataFrame(coefficients_dic_m)
print("Coefficients:", coefficients_m)
print("Intercept:", ridge_model_m.intercept_)
y_pred = ridge_model_m.predict(X)
mse = mean_squared_error(Y, y_pred)
print(f"Mean Squared Error: {mse}")

Lambda 5.32933066309083
Coefficients:                                         Players      Beta
0         United States of America_ian_sandoval  0.088933
1       United States of America_justin_ah chow  0.169553
2         United States of America_solen_chiodi -0.019617
3         United States of America_brandon_dang  0.124191
4        United States of America_caden_spencer -0.019617
..                                          ...       ...
158     United States of America_hunter_brunett -0.019617
159      United States of America_caden_peacock -0.019617
160     United States of America_cole_partridge  0.298425
161      United States of America_tyler_shimizu  0.069317
162  United States of America_frederick_richard -0.011971

[163 rows x 2 columns]
Intercept: 1.3974531805602837
Mean Squared Error: 2.4082374785250065


In [ ]:
ridge_betas = pd.concat([coefficients, coefficients_m]).reset_index(drop=True)
ridge_betas.to_csv("ridge_betas.csv")

In [ ]:
# get whole players beta

design_matrix_whole = pd.DataFrame(columns = whole["id"].unique())

competitions = whole["Competition"].unique()

competition_dict = dict()
for compet in competitions:
    data = whole[whole["Competition"] == compet]
    lst = list(data["id"].unique())
    data = data[data["Round"] != "qual"]
    data['Medal'] = np.where(data['Adjusted Rank'].isin([1.0,2.0,3.0]), 1, 0)
    medals = sum(data["Medal"])
    lst.append(medals)
    competition_dict[compet] = lst

i = 0
for compet in competitions:
    lst = competition_dict[compet]
    if len(lst) < 2:
        continue
    players = lst[0:-1]
    medals = lst[-1]
    for player in players:
        design_matrix_whole.at[i, player] = 1
    design_matrix_whole.at[i, "Medals"] = medals
    i += 1
design_matrix_whole = design_matrix_whole.fillna(0)

X = design_matrix_whole.drop("Medals", axis=1).values
Y = design_matrix_whole["Medals"].values

alphas = np.logspace(-6, 6, 10000)
ridge_cv_whole = RidgeCV(alphas=alphas, store_cv_values=True)
ridge_cv_whole.fit(X, Y)

best_alpha = ridge_cv_whole.alpha_

ridge_model_whole = RidgeCV(alphas=[best_alpha])
ridge_model_whole.fit(X, Y)

print("Lambda", best_alpha)
coefficients_dic_whole= {'id': list(set(whole["id"])),
                   'Beta': ridge_model_whole.coef_}
coefficients_whole = pd.DataFrame(coefficients_dic_whole)
print("Coefficients:", coefficients_whole)
print("Intercept:", ridge_model_whole.intercept_)
y_pred = ridge_model_whole.predict(X)
mse = mean_squared_error(Y, y_pred)
print(f"Mean Squared Error: {mse}")

Lambda 37.80574109020105
Coefficients:                                                id      Beta
0                           Croatia_marko_jovicic  0.013444
1                              China_jingyuan_zou -0.069253
2                          New Zealand_reece_cobb  0.036908
3                         France_mathias_philippe  0.018874
4                                 China_xinyi_sun  0.003122
...                                           ...       ...
1996                 Iceland_agust ingi_davidsson  0.004215
1997                     Hong Kong_cheuk lam_chan  0.013658
1998               Colombia_daira gisell_lamadrid  0.013658
1999  United States of America_kristian_grahovski  0.006601
2000                            Japan_kazuma_kaya  0.004215

[2001 rows x 2 columns]
Intercept: 22.484487036315112
Mean Squared Error: 0.594106012814139


In [ ]:
coefficients_whole.to_csv("betas_whole.csv", index = False)